In [ ]:
!nvidia-smi

In [ ]:
# google 掛載點位
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive') # 就是掛在這邊~

In [ ]:
!ls "/content/gdrive/MyDrive/ColabNotebooks"
# should colab.zip is exists.

In [ ]:
# !cp /content/gdrive/MyDrive/ColabNotebooks/* ./
# !ls

In [ ]:
# and copy some file from GD, like pt, or txt.
# !cp /content/gdrive/MyDrive/ColabNotebooks/1222_0009_08_e1500.pt ./
# !cp /content/gdrive/MyDrive/ColabNotebooks/loss_recoder_1221_1753_08.txt ./
# !ls

In [ ]:
!unzip -q /content/gdrive/MyDrive/ColabNotebooks/colab.zip -d ./

In [ ]:
# !ls
""" should list this files.
1221_0909_30_e999.pt   dataloader.py	 model.py	     train_local.ipynb
1221_1602_00_e1999.pt  gdrive		 rebuild_test.ipynb  train.py
after.png	       __init__.py	 res_AE.py	     unet
before.png	       loss_recoder.png  res_unet	     Untitled.ipynb
data		       lr_recoder.png	 sample_data	     util.py
"""

In [ ]:
import model as my_model
import dataloader as my_dataset
import train as my_train
from res_AE import ResNetAE  # res_ae = ResNetAE()


from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import random_split
import torch.nn as nn
from glob import glob

In [ ]:
class SaveBestModel:
    """
    Class to save the best model while training. If the current epoch's
    validation loss is less than the previous least less, then save the
    model state.
    """

    def __init__(
            self, save_root, best_valid_loss=float('inf')
    ):
        self.save_path = save_root
        ensure_dir(self.save_path)
        self.best_valid_loss = best_valid_loss

    def __call__(
            self, current_valid_loss, net, current_epoch,
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"\nBest validation loss: {self.best_valid_loss}")
            print(f"Saving best model for epoch: {current_epoch}\n")
            _file_name = Path(self.save_path).joinpath("best.pt")
            torch.save(net.state_dict(), _file_name)
            # for colab special command
            !cp /content/$_file_name /content/gdrive/MyDrive/ColabNotebooks/$_file_name
            return True
        else:
            return False

In [ ]:
good_bad_dataset = my_dataset.QR_good_bad_dataset("./data", image_size=256, image_channels=1)

# split dataset
#my_dataloader, my_val_dataset = random_split(good_bad_dataset, [0.7, 0.3])
my_dataloader, my_val_dataset, my_test_dataset = random_split(good_bad_dataset, [0.7, 0.2, 0.1],
                                                              generator=torch.Generator().manual_seed(42))
#my_dataloader, my_val_dataset, _ = random_split(good_bad_dataset, [7, 3, 720])  # for fast test

# 40GB: 32??
running_bs = 32  # batch size
my_dataloader = DataLoader(good_bad_dataset, batch_size=running_bs, drop_last=True, 
                           pin_memory=True, num_workers = 8)
my_val_dataloader = DataLoader(my_val_dataset, batch_size=running_bs, drop_last=True, 
                               pin_memory=True, num_workers = 8)
# 繪製 preview 用的大小
r, c = 5, 5
assert r * c <= len(my_test_dataset)
test_dataloader = DataLoader(my_test_dataset, batch_size=r*c, drop_last=False, pin_memory=False)
net = ResNetAE(input_shape=(256,256,1))
net.cuda()

#
torch.manual_seed(0)
# load model
# M_PATH = "1222_0009_08_e1500.pt"
# net.load_state_dict(torch.load(M_PATH))

In [ ]:
#!rm -r Runnings-save_1224_2215_47

In [ ]:
# 手糙 UTC-8
# from datetime import datetime, timedelta
# (datetime.now() + timedelta(hours=8)).strftime('%m%d_%H%M_%S')

In [ ]:
datetime.now()

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch import nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from tqdm import tqdm
import model as my_model
import dataloader as my_dataset
import torch.optim as optim
from datetime import datetime, timedelta
from torch.utils.data import random_split
from  util import ensure_dir
from pathlib import Path
# param
# net = net
dataloader=my_dataloader
val_dataloader=my_val_dataloader
epoches=20000
lr=1e-4  ########################################## 
SAVE_ROUND=50

# 時戳
current_time = lambda: (datetime.now() + timedelta(hours=8)).strftime('%m%d_%H%M_%S')

# 存檔目錄
SAVE_ROOT = Path(f"Runnings-save_{current_time()}")

# for colab special command
!mkdir /content/gdrive/MyDrive/ColabNotebooks/$SAVE_ROOT

ensure_dir(SAVE_ROOT)
_stamp = current_time()
LOSS_RECODER_TXT = SAVE_ROOT.joinpath(f"./loss_recoder_{_stamp}.txt")
with open(LOSS_RECODER_TXT, mode='a', encoding='utf-8') as f:
    f.write(f"=========== Start train: {_stamp} ===========\n")
    f.write(f"train loss,\t valid loss,\t lr\n")


cuda = torch.cuda.is_available()
assert cuda  # 必須為 == True, cuda 失敗
net = net.cuda()

optimizer = optim.Adam(net.parameters(), lr=lr)
# loss function
mse = nn.MSELoss()
# save 週期
save_round = SAVE_ROUND
save_cnt = 0
#
CosineAnnealingWarmRestarts = True
if CosineAnnealingWarmRestarts:
    alpha_ = epoches / 30  # hyp
    the_first_restart = int(epoches * (0.8 / alpha_))  # hyp
    T_mult = 2
    print(f"CosineAnnealingWarmRestarts: T_0:{the_first_restart}, T_mult:{T_mult}")
    train_scheduler = optim.lr_scheduler. \
        CosineAnnealingWarmRestarts(optimizer,
                                    T_0=the_first_restart,
                                    T_mult=T_mult, verbose=False)

# best saver
save_best = SaveBestModel(SAVE_ROOT)

loss_recoder = []
loss_val_recoder = []
lr_recoder = []
pbar = tqdm(range(epoches), smoothing=0.1, ncols=100)
for epoch in pbar:
    save_cnt += 1
    avg_batch_loss = 0
    avg_batch_val_loss = 0
    for batch_idx, data_good_bad in enumerate(dataloader):
        good, bad = data_good_bad
        good = good.cuda().float()/255.0
        bad = bad.cuda().float()/255.0
        bad = Variable(bad)
        optimizer.zero_grad()
        output = net(bad)
        loss = mse(output, good)
        avg_batch_loss += loss.detach().cpu().numpy()
        loss.backward()
        optimizer.step()
        if CosineAnnealingWarmRestarts:
            train_scheduler.step(epoch + batch_idx / len(dataloader))
            lr_recoder.append(train_scheduler.get_last_lr()[0])
    # end of dataloader
    # calc train loss
    __loss = avg_batch_loss / batch_idx
    loss_recoder.append(__loss)

    # calc val loss
    net.eval()
    for batch_idx, data_good_bad in enumerate(val_dataloader):
        good, bad = data_good_bad
        good = good.cuda().float() / 255.0
        bad = bad.cuda().float() / 255.0
        bad = Variable(bad)
        optimizer.zero_grad()
        output = net(bad)
        loss = mse(output, good)
        avg_batch_val_loss += loss.detach().cpu().numpy()
    #
    __val_loss = avg_batch_val_loss / batch_idx

    # save beset weight and preview
    if save_best(__val_loss, net, epoch):
        plt.style.use('classic')
        plt.imshow(my_train.make_test_view(net, test_dataloader,r,c, sub_size=128)*255.0, cmap='gray')
        _preview_save = SAVE_ROOT.joinpath(f"preview_{str(epoch).zfill(5)}.jpg")
        plt.savefig(_preview_save, dpi=300)
        !cp /content/$_preview_save /content/gdrive/MyDrive/ColabNotebooks/$_preview_save
        plt.clf()
        plt.cla()
        plt.close(plt.gcf())
    #
    loss_val_recoder.append(__val_loss)

    pbar.set_description("Training progress: Epoch{0} loss={1:.6f}, val_loss={2:.6f}".format(epoch + 1, __loss, __val_loss))
    # tranning save
    if save_cnt % save_round == 0:
        print("save model...")
        _pt_name = SAVE_ROOT.joinpath(f'{current_time()}_e{epoch+1}.pt')
        torch.save(net.state_dict(), _pt_name)
        #  for colab special command
        !cp /content/$_pt_name /content/gdrive/MyDrive/ColabNotebooks/$_pt_name
    # 紀錄 loss
    with open(LOSS_RECODER_TXT, mode='a', encoding='utf-8') as f:
        f.write(str(__loss) + ", " + str(__val_loss) + ", " + str(train_scheduler.get_last_lr()[0]) + "\n")
    # for colab special command
    !cp -f /content/$LOSS_RECODER_TXT /content/gdrive/MyDrive/ColabNotebooks/$LOSS_RECODER_TXT  
    #--------------------------------
    # loss
    plt.style.use('ggplot')
    fig = plt.figure("loss")
    plt.plot(loss_recoder, label='train loss')
    plt.plot(loss_val_recoder, label='valid loss')
    plt.legend()
    plt.yscale('log')
    # 
    loss_recoder_saven = SAVE_ROOT.joinpath("loss_recoder.png")
    fig.savefig(loss_recoder_saven, dpi=300)  # colab special
    !cp -f /content/$loss_recoder_saven /content/gdrive/MyDrive/ColabNotebooks/$loss_recoder_saven
    # lr
    fig_lr = plt.figure("lr")
    plt.plot(lr_recoder)
    plt.yscale('log')
    lr_recoder_saven = SAVE_ROOT.joinpath("lr_recoder.png")
    fig_lr.savefig(SAVE_ROOT.joinpath("lr_recoder.png"), dpi=300)
    !cp -f /content/$lr_recoder_saven /content/gdrive/MyDrive/ColabNotebooks/$lr_recoder_saven
    plt.clf()
    plt.cla()
    plt.close(fig_lr)
    plt.close(fig)
    plt.style.use('classic')

In [ ]:
!mkdir mkdir

In [ ]:
!mkdir result

In [ ]:
for test_idx in range(int(730*0.3)):
    #test_idx = 100
    # bad image
    test_batch = my_val_dataset[test_idx][1].reshape(1, 1, 256, 256)
    # show bad batch
    _test_batch_show = np.array(test_batch.reshape(256, 256))
    plt.imshow(_test_batch_show, cmap=plt.cm.gray)
    fig1 = plt.gcf()
    fig1.savefig("before.png")
    net.eval()
    output_batch = net(test_batch.cuda().float()/255.0) # / 255.0??

    # show output_batch batch
    _output_show = output_batch.reshape(256, 256).detach().cpu().numpy()
    plt.imshow(_output_show, cmap=plt.cm.gray)
    fig2 = plt.gcf()
    fig2.savefig("after.png")

    plt.close()

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle('Rebuild')
    #
    ax1.set_title('before')
    ax1.imshow(_test_batch_show, cmap=plt.cm.gray)
    #
    ax2.set_title('after')
    ax2.imshow(_output_show, cmap=plt.cm.gray)
    plt.tight_layout()
    fig.savefig(f"./result/rebuild_{test_idx}.png")
    #plt.show()

In [ ]:
#
loss_fn = "./loss_recoder_1221_1753_08.txt"

with open(loss_fn, mode='r', encoding='utf-8') as  f:
    lines = f.readlines()[2:]

tloss_list = []
vloss_list = []
lr_list = []
for line in lines:
    tloss, vloss, lr = line.rstrip().split(',')
    tloss_list.append(tloss)
    vloss_list.append(vloss)
    lr_list.append(lr)

import numpy as np
import matplotlib.pyplot as plt
tloss_list = np.array(tloss_list, dtype=np.float64)
vloss_list = np.array(vloss_list, dtype=np.float64)
lr_list = np.array(lr_list, dtype=np.float64)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))

ax.plot(tloss_list, label='train loss')
ax.plot(vloss_list, label='valid loss')
plt.xlabel('epoches', labelpad=10, fontsize=20)
plt.ylabel('loss', labelpad=10, fontsize=20)
ax.tick_params(axis='y')

ax.legend()

ax2 = ax.twinx()

ax2.plot(lr_list, color='gray', alpha=0.2, label='lr')
ax2.tick_params(axis='y', labelcolor='gray')
plt.ylabel('lr', labelpad=10, fontsize=20, rotation=0)

plt.legend()
plt.tight_layout()
plt.savefig("123.png", dpi=600)
plt.show()



In [ ]:
# net = my_train.train(net, my_dataloader, my_val_dataloader, epoches=20000, \
#                      lr=1e-5, SAVE_ROUND=50)

In [ ]:
# 打包資料夾，記得左邊把東西先拉到要下的資料夾內
!mkdir want_to_download

In [ ]:
# 打包資料夾，記得左邊把東西先拉到要下的資料夾內
!zip -r /content/result.zip /content/result


In [ ]:
# 打包東西拉到 GD
!cp -f /content/result.zip /content/gdrive/MyDrive/ColabNotebooks/result.zip

In [ ]:
# OK
# _pt_name = "model.py"
# def abc():
#     print('abc')
#     !cp /content/$_pt_name /content/gdrive/MyDrive/ColabNotebooks/$_pt_name
# abc()

In [ ]:
!cp /content/XXX.pt /content/gdrive/MyDrive/ColabNotebooks/XXX.pt